<a href="https://colab.research.google.com/github/nilsimda/bpc-project/blob/main/Train_HSB_DE_BPE_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Set up Repo

In [1]:
!git clone https://github.com/nilsimda/bpc-project.git

fatal: destination path 'bpc-project' already exists and is not an empty directory.


In [2]:
%cd bpc-project/

/content/bpc-project


In [ ]:
!uv sync

## 2. Prepare Dataset

In [6]:
!uv run ./prepare.sh bpe

Using segmentation method: bpe
Starting preprocessing for hsb-de with bpe...
Step 1-5: Processing train/dev/test splits through Moses pipeline...
Processing train split...
Tokenizer Version 1.1
Language: hsb
Number of threads: 1
Tokenizer Version 1.1
Language: de
Number of threads: 1
clean-corpus.perl: processing ./dataset/output_moses/train.tok.hsb & .de to ./dataset/output_moses/train.clean, cutoff 1-100, ratio 9
..........(100000)..........(200000)..........(300000)..........(400000)....
Input sentences: 449058  Output sentences:  449057
Processing dev split...
Tokenizer Version 1.1
Language: hsb
Number of threads: 1
Tokenizer Version 1.1
Language: de
Number of threads: 1
Processing test split...
Tokenizer Version 1.1
Language: hsb
Number of threads: 1
Tokenizer Version 1.1
Language: de
Number of threads: 1
Step 4: Training truecaser on training data...
Step 5: Applying truecasing to all splits...
Step 6: Learning BPE on training data...
Combined training data has 898114 lines
100% 

## 3. Train the model with Fairseq

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 && uv run fairseq-train \
  ./dataset/fairseq_bpe \
  --arch transformer_iwslt_de_en \
  --max-epoch 1 \
  --patience 10 \
  --save-interval 1 \
  --validate-interval 1 \
  --share-decoder-input-output-embed \
  --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
  --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --dropout 0.3 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
  --save-dir checkpoints/sorbian_german_bpe


2025-06-23 11:32:09 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': N

## 4. Evaluate the model

In [ ]:
!uv run ./eval.sh sorbian_german_bpe bpe dev